In [3]:
import os,sys, math, ROOT
from itertools import combinations
from array import array
import numpy as np
%jsroot on

#Importing a helper file where some functions are defined
from helper import *

#Input file
# inputfile = ROOT.TFile("events4leptonsCMS_FullRun2.root","READ")
inputfile = ROOT.TFile("Run2data_SkimFourLeptons.root","READ")

#Output file
outputfile = ROOT.TFile("output.root","RECREATE")

#Accessing the "TTree" from root that contains the data (one entry per event)
# tree = inputfile.Get("tree")
tree = inputfile.Get("ntuplizer/tree")

nentries = tree.GetEntries()
print("Number of entries: ", nentries)

Welcome to JupyROOT 6.28/00
Number of entries:  104317


In [15]:
#Example of histograms 
#The arguments are the following: 
#"name", "title;x axis title;yaxis title", nb of bins, min value, max value
hist_npv = ROOT.TH1F("hist_NPV", "NPV;Number of reconstructed vertices;Events", 100, 0, 100)
hist_massfourleptons = ROOT.TH1F("hist_massfourleptons",";Four Lepton Invariant Mass;Events", 125, 0, 500)

Warning in <TFile::Append>: Replacing existing TH1: hist_NPV (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hist_massfourleptons (Potential memory leak).


In [5]:
hist_4l = ROOT.TH1F("hist_4l",";4l;Events", 125, 0, 500)
hist_z1 = ROOT.TH1F("hist_z1",";z1;Events", 125, 0, 500)
hist_z2 = ROOT.TH1F("hist_z2",";z2;Events", 125, 0, 500)
#Loop over entries
nentries = tree.GetEntriesFast()
# nentries=50000
# nentries=30000

def Zmass(pair):
    z1 = pair[0]+pair[1]
    return abs(z1.M()-91)

for i in range(0, nentries):
    #Load entry number i
    tree.GetEntry(i)
    #Fill the number of reconstructed vertices in a histogram
    leps=[]
    if (len(tree._lPt)!=len(tree._lEta)): print(len(tree._lPt),len(tree._lEta))
    if (len(tree._lPt)!=len(tree._lPhi)): print(len(tree._lPt),len(tree._lPhi))
    if (len(tree._lPt)!=len(tree._lpdgId)): print(len(tree._lPt),len(tree._lpdgId))
    for ind in range(0,len(tree._lPt)):
        if abs(tree._lpdgId[ind])!=13: continue
        if tree._lPt[ind]<10: continue
        lep = ROOT.TLorentzVector()
        lep.SetPtEtaPhiM(tree._lPt[ind],tree._lEta[ind],tree._lPhi[ind],0)
        leps.append(lep)
    # if (tree._eventNb==602182346):
    #     print([(x.M(),x.Pt()) for x in leps])
    #     for ind in range(0,len(tree._lPt)):
    #         print(tree._lPt[ind],tree._lEta[ind],tree._lPhi[ind])
    # else: continue
    if(len(leps)<4): continue
    Z1 = leps[0]+leps[1]
    Z2 = leps[2]+leps[3]
    # print("pre",Z1.M(),Z2.M())
    min_pair = min(list(combinations(leps, 2)), key=Zmass)
    other_pair = list(set(leps)-set(min_pair))
    Z1 = min_pair[0]+min_pair[1]
    Z2 = other_pair[0]+other_pair[1]
    if Z1.M()<12 or Z2.M()<12: continue
    if Z1.M()<40 and Z2.M()<40: continue
    if Z1.M()>120 or Z2.M()>120: continue
    # print("post", tree._eventNb, Z1.M(),Z2.M())
    H = Z1 + Z2
    hist_4l.Fill(tree._m4l)
    hist_z1.Fill(Z1.M())
    hist_z2.Fill(Z2.M())

c = ROOT.TCanvas("canvas","c_NPV",600,600)
#Draw the histogram and the canvas
hist_4l.Draw()
cname="c_4m_sort"
name="cut"
c.SaveAs(cname+"_4l_"+name+".pdf")
hist_z1.Draw()
c.SaveAs(cname+"_z1_"+name+".pdf")
hist_z2.Draw()
c.SaveAs(cname+"_z2_"+name+".pdf")
c.Close()

Warning in <TFile::Append>: Replacing existing TH1: hist_4l (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hist_z1 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hist_z2 (Potential memory leak).
Error in <TBasket::Streamer>: The value of fIOBits (00000000000000000000000001110001) contains unknown flags (supported flags are 00000000000000000000000000000001), indicating this was written with a newer version of ROOT utilizing critical IO features this version of ROOT does not support.  Refusing to deserialize.
Error in <TBasket::Streamer>: The value of fIOBits (00000000000000000000000001110001) contains unknown flags (supported flags are 00000000000000000000000000000001), indicating this was written with a newer version of ROOT utilizing critical IO features this version of ROOT does not support.  Refusing to deserialize.
Error in <TBasket::Streamer>: The value of fIOBits (00000000000000000000000001110001) contains unknown flags 

In [20]:
c = ROOT.TCanvas("canvas","c_NPV",600,600)
#Draw the histogram and the canvas
hist_massfourleptons.Draw()
c.Draw()
c.SaveAs("canvas.pdf")
c.Close()

Info in <TCanvas::Print>: pdf file canvas.pdf has been created


Info in <TCanvas::Print>: pdf file canvas.pdf has been created


In [9]:
savehisto(outputfile, hist_massfourleptons, "hist_MassFourLeptons")
hist_massfourleptons.Draw()
c.Draw()

Info in <TCanvas::Print>: png file c_hist_MassFourLeptons.png has been created
